In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow_datasets as tfds
import os

In [ ]:
# hyper-parameters
batch_size = 64
# 10 categories of images (CIFAR-10)
num_classes = 10
# number of training epochs
epochs = 30

In [ ]:
# CIFAR-10 daataset has 10 classes 50000 samples for training and 10000 samples for testing
# image 32x32x3 pixels (width and height = 32 depth = 3)

In [ ]:
def load_data():
  #load CIFAR-10 dataset and preprocess it
  def preprocess_image(image,label):
    # convert [0,255] range integers to [0,1] range floats
    image = tf.image.convert_image_dtype(image,tf.float32)
    return image, label
  # split dataset as train and test sets
  ds_train, info = tfds.load("cifar10",with_info=True,split="train",as_supervised=True)
  ds_test = tfds.load("cifar10",split="test",as_supervised=True)
  # repeat 
  ds_train = ds_train.repeat().shuffle(1024).map(preprocess_image).batch(batch_size)
  ds_test = ds_test.repeat().shuffle(1024).map(preprocess_image).batch(batch_size)
  return ds_train, ds_test, info

In [ ]:
# with_info = True is using for get info of datasets
# repeat dataset forever with repeat() (will specify stopping conditions in the training phase)
# 64 samples use with batch()

In [ ]:
def create_model(input_shape):
    # building the model
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # flattening the convolutions
    model.add(Flatten())
    # fully-connected layer
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))
    # print the summary of the model architecture
    model.summary()
    # training the model using adam optimizer
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [ ]:
# training model
if __name__ == "__main__":
  #load data
  ds_train,ds_test,info = load_data()
  model = create_model(input_shape = info.features["image"].shape)
  # some nice callbacks
  logdir = os.path.join("logs","cifar10-model-v1")
  tensorboard = TensorBoard(log_dir=logdir)
  # make sure resulst folder exist
  if not os.path.isdir("result"):
    os.mkdir("results")
  #train
  model.fit(ds_train,epochs=epochs,validation_data=ds_test,verbose=1,
            steps_per_epoch=info.splits["train"].num_examples//batch_size,
            validation_steps=info.splits["test"].num_examples//batch_size,
            callbacks=[tensorboard])
  # save the model to disk
  model.save("results/cifar10-model-v1.h5")
  

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incompleteYZ7J81/cifar10-train.tfrecord*...:   0%|          | 0/…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incompleteYZ7J81/cifar10-test.tfrecord*...:   0%|          | 0/1…

Dataset cifar10 downloaded and prepared to ~/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dr